# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f5374f0efa0>
├── 'a' --> tensor([[-1.1131,  1.1599, -1.0047],
│                   [ 0.2957,  0.9710, -0.5843]])
└── 'x' --> <FastTreeValue 0x7f5374f0e160>
    └── 'c' --> tensor([[ 1.3360, -1.8095,  0.5753, -0.8069],
                        [-1.0308,  0.4240, -0.6239, -0.2926],
                        [ 0.0912, -0.0262, -0.0694,  2.1092]])

In [4]:
t.a

tensor([[-1.1131,  1.1599, -1.0047],
        [ 0.2957,  0.9710, -0.5843]])

In [5]:
%timeit t.a

83.7 ns ± 0.614 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f5374f0efa0>
├── 'a' --> tensor([[-0.9958, -0.4824,  0.3217],
│                   [-0.6671,  0.3794, -0.6759]])
└── 'x' --> <FastTreeValue 0x7f5374f0e160>
    └── 'c' --> tensor([[ 1.3360, -1.8095,  0.5753, -0.8069],
                        [-1.0308,  0.4240, -0.6239, -0.2926],
                        [ 0.0912, -0.0262, -0.0694,  2.1092]])

In [7]:
%timeit t.a = new_value

89.2 ns ± 0.891 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.1131,  1.1599, -1.0047],
               [ 0.2957,  0.9710, -0.5843]]),
    x: Batch(
           c: tensor([[ 1.3360, -1.8095,  0.5753, -0.8069],
                      [-1.0308,  0.4240, -0.6239, -0.2926],
                      [ 0.0912, -0.0262, -0.0694,  2.1092]]),
       ),
)

In [10]:
b.a

tensor([[-1.1131,  1.1599, -1.0047],
        [ 0.2957,  0.9710, -0.5843]])

In [11]:
%timeit b.a

70.7 ns ± 0.824 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.9120, -1.3238, -1.9095],
               [ 0.8458, -0.5200,  1.8140]]),
    x: Batch(
           c: tensor([[ 1.3360, -1.8095,  0.5753, -0.8069],
                      [-1.0308,  0.4240, -0.6239, -0.2926],
                      [ 0.0912, -0.0262, -0.0694,  2.1092]]),
       ),
)

In [13]:
%timeit b.a = new_value

652 ns ± 5.27 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.14 µs ± 18.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

15.1 µs ± 91.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

274 µs ± 10.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

255 µs ± 8.31 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f5374e98340>
├── 'a' --> tensor([[[-1.1131,  1.1599, -1.0047],
│                    [ 0.2957,  0.9710, -0.5843]],
│           
│                   [[-1.1131,  1.1599, -1.0047],
│                    [ 0.2957,  0.9710, -0.5843]],
│           
│                   [[-1.1131,  1.1599, -1.0047],
│                    [ 0.2957,  0.9710, -0.5843]],
│           
│                   [[-1.1131,  1.1599, -1.0047],
│                    [ 0.2957,  0.9710, -0.5843]],
│           
│                   [[-1.1131,  1.1599, -1.0047],
│                    [ 0.2957,  0.9710, -0.5843]],
│           
│                   [[-1.1131,  1.1599, -1.0047],
│                    [ 0.2957,  0.9710, -0.5843]],
│           
│                   [[-1.1131,  1.1599, -1.0047],
│                    [ 0.2957,  0.9710, -0.5843]],
│           
│                   [[-1.1131,  1.1599, -1.0047],
│                    [ 0.2957,  0.9710, -0.5843]]])
└── 'x' --> <FastTreeValue 0x7f5374e98820>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

46.5 µs ± 796 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f52be146100>
├── 'a' --> tensor([[-1.1131,  1.1599, -1.0047],
│                   [ 0.2957,  0.9710, -0.5843],
│                   [-1.1131,  1.1599, -1.0047],
│                   [ 0.2957,  0.9710, -0.5843],
│                   [-1.1131,  1.1599, -1.0047],
│                   [ 0.2957,  0.9710, -0.5843],
│                   [-1.1131,  1.1599, -1.0047],
│                   [ 0.2957,  0.9710, -0.5843],
│                   [-1.1131,  1.1599, -1.0047],
│                   [ 0.2957,  0.9710, -0.5843],
│                   [-1.1131,  1.1599, -1.0047],
│                   [ 0.2957,  0.9710, -0.5843],
│                   [-1.1131,  1.1599, -1.0047],
│                   [ 0.2957,  0.9710, -0.5843],
│                   [-1.1131,  1.1599, -1.0047],
│                   [ 0.2957,  0.9710, -0.5843]])
└── 'x' --> <FastTreeValue 0x7f535063a2b0>
    └── 'c' --> tensor([[ 1.3360, -1.8095,  0.5753, -0.8069],
                        [-1.0308,  0.4240, -0.6239, -0.2926],
                 

In [23]:
%timeit t_cat(trees)

43.7 µs ± 924 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

83.7 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.1131,  1.1599, -1.0047],
                [ 0.2957,  0.9710, -0.5843]],
       
               [[-1.1131,  1.1599, -1.0047],
                [ 0.2957,  0.9710, -0.5843]],
       
               [[-1.1131,  1.1599, -1.0047],
                [ 0.2957,  0.9710, -0.5843]],
       
               [[-1.1131,  1.1599, -1.0047],
                [ 0.2957,  0.9710, -0.5843]],
       
               [[-1.1131,  1.1599, -1.0047],
                [ 0.2957,  0.9710, -0.5843]],
       
               [[-1.1131,  1.1599, -1.0047],
                [ 0.2957,  0.9710, -0.5843]],
       
               [[-1.1131,  1.1599, -1.0047],
                [ 0.2957,  0.9710, -0.5843]],
       
               [[-1.1131,  1.1599, -1.0047],
                [ 0.2957,  0.9710, -0.5843]]]),
    x: Batch(
           c: tensor([[[ 1.3360, -1.8095,  0.5753, -0.8069],
                       [-1.0308,  0.4240, -0.6239, -0.2926],
                       [ 0.0912, -0.0262, -0.0694,  2.1092]],
         

In [26]:
%timeit Batch.stack(batches)

110 µs ± 1.67 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.1131,  1.1599, -1.0047],
               [ 0.2957,  0.9710, -0.5843],
               [-1.1131,  1.1599, -1.0047],
               [ 0.2957,  0.9710, -0.5843],
               [-1.1131,  1.1599, -1.0047],
               [ 0.2957,  0.9710, -0.5843],
               [-1.1131,  1.1599, -1.0047],
               [ 0.2957,  0.9710, -0.5843],
               [-1.1131,  1.1599, -1.0047],
               [ 0.2957,  0.9710, -0.5843],
               [-1.1131,  1.1599, -1.0047],
               [ 0.2957,  0.9710, -0.5843],
               [-1.1131,  1.1599, -1.0047],
               [ 0.2957,  0.9710, -0.5843],
               [-1.1131,  1.1599, -1.0047],
               [ 0.2957,  0.9710, -0.5843]]),
    x: Batch(
           c: tensor([[ 1.3360, -1.8095,  0.5753, -0.8069],
                      [-1.0308,  0.4240, -0.6239, -0.2926],
                      [ 0.0912, -0.0262, -0.0694,  2.1092],
                      [ 1.3360, -1.8095,  0.5753, -0.8069],
                      [-1.0308,  

In [28]:
%timeit Batch.cat(batches)

195 µs ± 1.32 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

546 µs ± 12.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
